In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mbti-type/mbti_1.csv


In [3]:
data = pd.read_csv("../input/mbti-type/mbti_1.csv")

In [4]:
data.head()

,type,posts
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1,ENTP,'I'm finding the lack of me in these posts ver...
2,INTP,'Good one _____ https://www.youtube.com/wat...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o..."
4,ENTJ,'You're fired.|||That's another silly misconce...


The first thing I want to do is turn the personality types into numbered categories, ranging from 0 to 15. These will serve as the output of the model when it is categorizing people.

In [5]:
types = np.unique(data.type.values)

In [6]:
def get_type_index(string):
    return list(types).index(string)

In [7]:
data['type_index'] = data['type'].apply(get_type_index)

In [8]:
data.posts.values[0]

"'http://www.youtube.com/watch?v=qsXHcwe3krw|||http://41.media.tumblr.com/tumblr_lfouy03PMA1qa1rooo1_500.jpg|||enfp and intj moments  https://www.youtube.com/watch?v=iz7lE1g4XM4  sportscenter not top ten plays  https://www.youtube.com/watch?v=uCdfze1etec  pranks|||What has been the most life-changing experience in your life?|||http://www.youtube.com/watch?v=vXZeYwwRDw8   http://www.youtube.com/watch?v=u8ejam5DP3E  On repeat for most of today.|||May the PerC Experience immerse you.|||The last thing my INFJ friend posted on his facebook before committing suicide the next day. Rest in peace~   http://vimeo.com/22842206|||Hello ENFJ7. Sorry to hear of your distress. It's only natural for a relationship to not be perfection all the time in every moment of existence. Try to figure the hard times as times of growth, as...|||84389  84390  http://wallpaperpassion.com/upload/23700/friendship-boy-and-girl-wallpaper.jpg  http://assets.dornob.com/wp-content/uploads/2010/04/round-home-design.jpg ...

Now, I want to clean the text to get rid of hyperlinks, puncuation, and anything else that's cluttering up the text. Specifically, the pipe (|) character seems like it separates different posts, but without spaces between them. First, I'm going to replace the pipes with spaces so that the tokenizer won't parse those parts as one long word. Then, I'll clean up the rest.

In [9]:
import string
import re

def clean_text(text):
    regex = re.compile('[%s]' % re.escape('|'))
    text = regex.sub(" ", text)
    words = str(text).split()
    words = [i.lower() + " " for i in words]
    words = [i for i in words if not "http" in i]
    words = " ".join(words)
    words = words.translate(words.maketrans('', '', string.punctuation))
    return words


In [10]:
data['cleaned_text'] = data['posts'].apply(clean_text)

In [11]:
data.cleaned_text.values[0]

'enfp  and  intj  moments  sportscenter  not  top  ten  plays  pranks  what  has  been  the  most  lifechanging  experience  in  your  life  on  repeat  for  most  of  today  may  the  perc  experience  immerse  you  the  last  thing  my  infj  friend  posted  on  his  facebook  before  committing  suicide  the  next  day  rest  in  peace  hello  enfj7  sorry  to  hear  of  your  distress  its  only  natural  for  a  relationship  to  not  be  perfection  all  the  time  in  every  moment  of  existence  try  to  figure  the  hard  times  as  times  of  growth  as  84389  84390    welcome  and  stuff  game  set  match  prozac  wellbrutin  at  least  thirty  minutes  of  moving  your  legs  and  i  dont  mean  moving  them  while  sitting  in  your  same  desk  chair  weed  in  moderation  maybe  try  edibles  as  a  healthier  alternative  basically  come  up  with  three  items  youve  determined  that  each  type  or  whichever  types  you  want  to  do  would  more  than  likely  us

In [12]:
data.head()

,type,posts,type_index,cleaned_text
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...,8,enfp and intj moments sportscenter not t...
1,ENTP,'I'm finding the lack of me in these posts ver...,3,im finding the lack of me in these pos...
2,INTP,'Good one _____ https://www.youtube.com/wat...,11,good one of course to which i say i ...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o...",10,dear intp i enjoyed our conversation the...
4,ENTJ,'You're fired.|||That's another silly misconce...,2,youre fired thats another silly misconcep...


Now, we split the data into training, testing, and validation sets,

In [13]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data)
train, val = train_test_split(train)

Keras has a great tokenizer that we can use to turn sequences of words into arrays of numbers. For more information, see: https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer

In [14]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

vocab_size = 10000
trunc_type = "post"
pad_type = "post"
oov_tok = "<OOV>"
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(data.cleaned_text.values)

In [15]:
maxlen = 1500
train_sequences = tokenizer.texts_to_sequences(train.cleaned_text.values)
train_padded = pad_sequences(train_sequences, maxlen = maxlen, truncating = trunc_type, padding = pad_type)

val_sequences = tokenizer.texts_to_sequences(val.cleaned_text.values)
val_padded = pad_sequences(val_sequences, maxlen = maxlen, truncating = trunc_type, padding = pad_type)


In [16]:
train_padded

array([[  48, 3702, 7640, ...,    0,    0,    0],
       [   3,  192,    9, ...,    0,    0,    0],
       [  62,  233,  576, ...,    0,    0,    0],
       ...,
       [ 560, 7318,  149, ...,    0,    0,    0],
       [   2,  157,   45, ...,    0,    0,    0],
       [ 783,  605,    3, ...,   45,  103,    9]], dtype=int32)

So, our model is going to take in these arrays of numbers that represent the text, and it's going to output the personality type that it thinks is associated with it. Here, I'm going to convert the personality types to one-hot-encoded labels. This simply means that to represent a particular category, we make an array with the length of the total possible number of categories, and make all of the values zero except at the index of the category we're trying to represent. 

In [17]:
one_hot_labels = tf.keras.utils.to_categorical(train.type_index.values, num_classes=16)
val_labels= tf.keras.utils.to_categorical(val.type_index.values, num_classes=16)

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Flatten, Dropout, Conv1D, GlobalMaxPooling1D

def create_model():
    op = tf.keras.optimizers.Adam(learning_rate=0.00001)

    model = Sequential()
    model.add(Embedding(vocab_size, 256, input_length=maxlen-1))
    model.add(Dropout(0.3))
    model.add(Bidirectional(LSTM(200, return_sequences=True)))
    model.add(Dropout(0.3))
    model.add(Bidirectional(LSTM(20)))
    model.add(Dropout(0.3))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(16, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=op, metrics=['accuracy'])
    return model

Using a TPU can greatly reduce the amount of time spent training the model.

In [19]:
use_tpu = True
if use_tpu:
    # Create distribution strategy
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)

    # Create model
    with strategy.scope():
        model = create_model()
else:
    model = create_model()
    
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1499, 256)         2560000   
_________________________________________________________________
dropout (Dropout)            (None, 1499, 256)         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 1499, 400)         731200    
_________________________________________________________________
dropout_1 (Dropout)          (None, 1499, 400)         0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 40)                67360     
_________________________________________________________________
dropout_2 (Dropout)          (None, 40)                0         
_________________________________________________________________
dense (Dense)                (None, 64)                2

In [20]:
model.fit(train_padded, one_hot_labels, epochs =20, verbose = 1, 
          validation_data = (val_padded, val_labels),  callbacks = [tf.keras.callbacks.EarlyStopping(patience = 3)])

Epoch 1/20
153/153 [==============================] - 34s 225ms/step - loss: 2.7681 - accuracy: 0.0953 - val_loss: 2.7551 - val_accuracy: 0.2090
Epoch 2/20
153/153 [==============================] - 30s 193ms/step - loss: 2.7379 - accuracy: 0.1970 - val_loss: 2.7033 - val_accuracy: 0.2151
Epoch 3/20
153/153 [==============================] - 30s 193ms/step - loss: 2.6606 - accuracy: 0.2033 - val_loss: 2.5631 - val_accuracy: 0.2151
Epoch 4/20
153/153 [==============================] - 30s 193ms/step - loss: 2.5707 - accuracy: 0.1941 - val_loss: 2.5129 - val_accuracy: 0.2151
Epoch 5/20
153/153 [==============================] - 30s 193ms/step - loss: 2.5328 - accuracy: 0.1882 - val_loss: 2.4780 - val_accuracy: 0.2151
Epoch 6/20
153/153 [==============================] - 30s 193ms/step - loss: 2.4800 - accuracy: 0.2068 - val_loss: 2.4115 - val_accuracy: 0.2151
Epoch 7/20
153/153 [==============================] - 30s 193ms/step - loss: 2.4373 - accuracy: 0.2029 - val_loss: 2.3690 - val_ac

This model didn't do very well, only achieving around 20% accuracy. This is a difficult challenge- to classify people into 16 different categories based on text that may loosely correlate with those categories. Let's see if we can do a bit better by incorporating a transformer. I used the one from this Keras example: https://keras.io/examples/nlp/text_classification_with_transformer/

For more information about transformers in general, see: https://medium.com/inside-machine-learning/what-is-a-transformer-d07dd1fbec04

In [21]:
from tensorflow.keras import layers
from tensorflow import keras
class MultiHeadSelfAttention(layers.Layer):
    def __init__(self, embed_dim, num_heads=8):
        super(MultiHeadSelfAttention, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        if embed_dim % num_heads != 0:
            raise ValueError(
                f"embedding dimension = {embed_dim} should be divisible by number of heads = {num_heads}"
            )
        self.projection_dim = embed_dim // num_heads
        self.query_dense = layers.Dense(embed_dim)
        self.key_dense = layers.Dense(embed_dim)
        self.value_dense = layers.Dense(embed_dim)
        self.combine_heads = layers.Dense(embed_dim)

    def attention(self, query, key, value):
        score = tf.matmul(query, key, transpose_b=True)
        dim_key = tf.cast(tf.shape(key)[-1], tf.float32)
        scaled_score = score / tf.math.sqrt(dim_key)
        weights = tf.nn.softmax(scaled_score, axis=-1)
        output = tf.matmul(weights, value)
        return output, weights

    def separate_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        # x.shape = [batch_size, seq_len, embedding_dim]
        batch_size = tf.shape(inputs)[0]
        query = self.query_dense(inputs)  # (batch_size, seq_len, embed_dim)
        key = self.key_dense(inputs)  # (batch_size, seq_len, embed_dim)
        value = self.value_dense(inputs)  # (batch_size, seq_len, embed_dim)
        query = self.separate_heads(
            query, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        key = self.separate_heads(
            key, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        value = self.separate_heads(
            value, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        attention, weights = self.attention(query, key, value)
        attention = tf.transpose(
            attention, perm=[0, 2, 1, 3]
        )  # (batch_size, seq_len, num_heads, projection_dim)
        concat_attention = tf.reshape(
            attention, (batch_size, -1, self.embed_dim)
        )  # (batch_size, seq_len, embed_dim)
        output = self.combine_heads(
            concat_attention
        )  # (batch_size, seq_len, embed_dim)
        return output

In [22]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadSelfAttention(embed_dim, num_heads)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [23]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, emded_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=emded_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=emded_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions


In [24]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer

def create_model(): 
    inputs = layers.Input(shape=(maxlen,))
    embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
    x = embedding_layer(inputs)
    transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
    x = transformer_block(x)
    # x = layers.GlobalAveragePooling1D()(x)
    # x = layers.Dropout(0.1)(x)
    x = (Bidirectional(LSTM(200, return_sequences=True)))(x)
    x = (Dropout(0.3))(x)
    x = (Bidirectional(LSTM(20)))(x)
    x = layers.Dense(20, activation="relu")(x)
    x = layers.Dropout(0.1)(x)
    outputs = layers.Dense(16, activation="softmax")(x)
    
    op = tf.keras.optimizers.Adam(learning_rate=0.00001)

    model = keras.Model(inputs=inputs, outputs=outputs)
    model.compile(op, 'categorical_crossentropy', metrics = ['accuracy'])
    return model

In [25]:
use_tpu = True
if use_tpu:
    # Create distribution strategy
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)

    # Create model
    with strategy.scope():
        model = create_model()
else:
    model = create_model()
    
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1500)]            0         
_________________________________________________________________
token_and_position_embedding (None, 1500, 32)          368000    
_________________________________________________________________
transformer_block (Transform (None, 1500, 32)          6464      
_________________________________________________________________
bidirectional_2 (Bidirection (None, 1500, 400)         372800    
_________________________________________________________________
dropout_5 (Dropout)          (None, 1500, 400)         0         
_________________________________________________________________
bidirectional_3 (Bidirection (None, 40)                67360     
_________________________________________________________________
dense_8 (Dense)              (None, 20)                820   

In [26]:
model.fit(train_padded, one_hot_labels, epochs =30, verbose = 1, 
          validation_data = (val_padded, val_labels), callbacks = [tf.keras.callbacks.EarlyStopping(patience = 3)])

Epoch 1/30
153/153 [==============================] - 32s 212ms/step - loss: 2.7164 - accuracy: 0.0611 - val_loss: 2.6803 - val_accuracy: 0.1057
Epoch 2/30
153/153 [==============================] - 26s 169ms/step - loss: 2.6551 - accuracy: 0.1804 - val_loss: 2.6016 - val_accuracy: 0.2004
Epoch 3/30
153/153 [==============================] - 26s 168ms/step - loss: 2.5917 - accuracy: 0.1972 - val_loss: 2.5489 - val_accuracy: 0.2163
Epoch 4/30
153/153 [==============================] - 26s 168ms/step - loss: 2.5554 - accuracy: 0.1941 - val_loss: 2.5129 - val_accuracy: 0.2151
Epoch 5/30
153/153 [==============================] - 26s 169ms/step - loss: 2.5305 - accuracy: 0.1984 - val_loss: 2.4887 - val_accuracy: 0.2151
Epoch 6/30
153/153 [==============================] - 26s 168ms/step - loss: 2.5155 - accuracy: 0.1923 - val_loss: 2.4711 - val_accuracy: 0.2151
Epoch 7/30
153/153 [==============================] - 26s 168ms/step - loss: 2.5013 - accuracy: 0.1986 - val_loss: 2.4582 - val_ac

In [27]:
import transformers
tokenizer = transformers.AutoTokenizer.from_pretrained('bert-large-uncased')

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [28]:
maxlen = 1500

train_input_ids = [tokenizer.encode(str(i), max_length = maxlen , pad_to_max_length = True) for i in train.cleaned_text.values]
val_input_ids = [tokenizer.encode(str(i), max_length = maxlen , pad_to_max_length = True) for i in val.cleaned_text.values]

In [29]:
def create_model(): 
    input_word_ids = tf.keras.layers.Input(shape=(maxlen,), dtype=tf.int32,
                                           name="input_word_ids")
    bert_layer = transformers.TFBertModel.from_pretrained('bert-large-uncased')
    bert_outputs = bert_layer(input_word_ids)[0]
    pred = tf.keras.layers.Dense(16, activation='softmax')(bert_outputs[:,0,:])
    
    model = tf.keras.models.Model(inputs=input_word_ids, outputs=pred)
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(
    learning_rate=0.00001), metrics=['accuracy'])
    return model

In [30]:
use_tpu = True
if use_tpu:
    # Create distribution strategy
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)

    # Create model
    with strategy.scope():
        model = create_model()
else:
    model = create_model()
    
model.summary()


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_word_ids (InputLayer)  [(None, 1500)]            0         
_________________________________________________________________
tf_bert_model (TFBertModel)  ((None, 1500, 1024), (Non 335141888 
_________________________________________________________________
tf_op_layer_strided_slice (T [(None, 1024)]            0         
_________________________________________________________________
dense_10 (Dense)             (None, 16)                16400     
Total params: 335,158,288
Trainable params: 335,158,288
Non-trainable params: 0
_________________________________________________________________


In [31]:
batch_size = 16

model.fit(np.array(train_input_ids), one_hot_labels,validation_data = (np.array(val_input_ids), val_labels),
          verbose = 1, epochs = 20, batch_size = batch_size,  callbacks = [tf.keras.callbacks.EarlyStopping(patience = 5)])

Epoch 1/20
305/305 [==============================] - 252s 827ms/step - loss: 2.2753 - accuracy: 0.2466 - val_loss: 1.9208 - val_accuracy: 0.4173
Epoch 2/20
305/305 [==============================] - 227s 746ms/step - loss: 1.6576 - accuracy: 0.5071 - val_loss: 1.4305 - val_accuracy: 0.5753
Epoch 3/20
305/305 [==============================] - 227s 745ms/step - loss: 1.3358 - accuracy: 0.6055 - val_loss: 1.2246 - val_accuracy: 0.6269
Epoch 4/20
305/305 [==============================] - 227s 745ms/step - loss: 1.1692 - accuracy: 0.6415 - val_loss: 1.1644 - val_accuracy: 0.6423
Epoch 5/20
305/305 [==============================] - 227s 745ms/step - loss: 1.0331 - accuracy: 0.6907 - val_loss: 1.2339 - val_accuracy: 0.6318
Epoch 6/20
305/305 [==============================] - 227s 745ms/step - loss: 0.9202 - accuracy: 0.7163 - val_loss: 1.1801 - val_accuracy: 0.6454
Epoch 7/20
305/305 [==============================] - 227s 745ms/step - loss: 0.7643 - accuracy: 0.7674 - val_loss: 1.3042 -

In [32]:
test_input_ids = [tokenizer.encode(str(i), max_length = maxlen , pad_to_max_length = True) for i in test.cleaned_text.values]
test_labels= tf.keras.utils.to_categorical(test.type_index.values, num_classes=16)

In [33]:
model.evaluate(np.array(test_input_ids), test_labels)

68/68 [==============================] - 20s 292ms/step - loss: 1.3413 - accuracy: 0.6487


[1.3412909507751465, 0.6486860513687134]